## Reading Geojson data using geopandas
### Tejas Choudekar

In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
#pip install folium matplotlib mapclassify
import geopandas as gp
from shapely.geometry import LineString, shape, Point, Polygon
import fiona
import matplotlib
# pip install fastai
from fastai.imports import *
import gtfs_kit as gk
import folium
from geojson import Feature, FeatureCollection, Point
from obspy.geodetics import degrees2kilometers
from geopy import distance

ModuleNotFoundError: No module named 'obspy'

In [2]:
pip install obspy

     |████████████████████████████████| 14.0 MB 2.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Reading geojson file

In [ ]:
geo_data = gp.read_file('eixodevia.geojson', ignore_geometry=False, encoding = 'utf-8')

# display(geo_data)

### Plotting map with all the route ids

In [ ]:
geo_data.explore(column='Rua Id',cmap='Set2')

### Reading and validating GTFS files

In [ ]:
#Declare the directory path for the GTFS zip file
path = Path("cascais.zip")

#Read the feed with gtfs-kit
feed = (gk.read_feed(path, dist_units='km'))

#Search for errors and warnings in the feed
feed.validate()

### Plotting bus stops of Cascias using GTFS files

In [ ]:


import warnings
warnings.filterwarnings('ignore')

#Check for errors in stops by plotting them to a map
feed.map_stops(feed.stops.stop_id[:])

### Plotting bus routes using GTFS files

In [ ]:
#Check for errors in routes by plotting them to a map
feed.map_routes(feed.routes.route_id.iloc[:], include_stops=True)

### Converting busroutes and stops to Geojson data to visualise better

In [ ]:
bus_routes = feed.routes_to_geojson(feed.routes.route_id.iloc[:], include_stops=True)

In [ ]:
bus_rou_gp = gp.GeoDataFrame.from_features(bus_routes)

### Filtering geojson data only for one street named Avenida Vinte e Cinco de Abril

In [ ]:

A2DA_geo_data = geo_data[geo_data.Nome == 'Avenida Vinte e Cinco de Abril']

In [ ]:
# A2DA_geo_data

### Visualising the selected street and number of bus networks passing through that street

In [ ]:
m_routes_filtered_A2DA = A2DA_geo_data.explore(column='Rua Id',cmap='Set2')
bus_rou_gp.explore(m=m_routes_filtered_A2DA, color="red")
folium.TileLayer('Stamen Toner', control=True).add_to(m_routes_filtered_A2DA)  # use folium to add alternative tiles
folium.LayerControl().add_to(m_routes_filtered_A2DA)  # use folium to add layer control

m_routes_filtered_A2DA  # show map

### Filtering the geojson data only to street of our interest and some area around it

In [ ]:
interest_geo_data = geo_data.cx[-9.4375:-9.4125, 38.69:38.71]

In [ ]:
interest_geo_data.plot(figsize=(15, 15))


In [ ]:
# interest_geo_data

### Plotting the intersections of the street using below get_intersections function

In [ ]:
def get_intersections(gdf, name, geom = 'geometry', crs = 4326):
    intersections_gdf = pd.DataFrame()
    name_2 = '{}_2'.format(name)
    crs_init = {'init': 'epsg:{}'.format(crs)}
    for index, row in gdf.iterrows():
        row_intersections = gdf.intersection(row[geom])
        row_intersection_points = row_intersections[row_intersections.geom_type == 'Point']
        row_intersections_df = pd.DataFrame(row_intersection_points)
        row_intersections_df[name_2] = row[name]
        row_intersections_df = row_intersections_df.join(gdf[name])
        intersections_gdf = row_intersections_df.append(intersections_gdf)

    intersections_gdf = intersections_gdf.rename(columns={0: geom})
    intersections_gdf['intersection'] = intersections_gdf.apply(lambda row: '-'.join(sorted([row[name_2], row[name]])), axis = 1)
    intersections_gdf = intersections_gdf.groupby('intersection').first()
    intersections_gdf = intersections_gdf.reset_index()
    intersections_gdf = gp.GeoDataFrame(intersections_gdf, geometry = 'geometry')
    intersections_gdf.crs = crs_init

    return intersections_gdf

In [ ]:
route_intersections = get_intersections(interest_geo_data, 'Rua Id')

In [ ]:
# route_intersections

In [ ]:
# route_intersections.plot(figsize=(15, 15))

In [ ]:
fig, ax = plt.subplots(figsize = (15,15))
route_fig = interest_geo_data.plot(ax = ax, color = 'darkgray', label = 'Routes')
intersect_points = route_intersections.plot(ax = ax, color = 'black', label = 'Route Intersections')
ax.legend(shadow=True, fancybox=True)
ax.set_title('Area of interest')
plt.show()

In [ ]:
# route_intersections.explore()

m_routes_intersection = route_intersections.explore(color="black", legend = True)
interest_geo_data.explore(m=m_routes_intersection, color="red")
folium.TileLayer('Stamen Toner', control=True).add_to(m_routes_intersection)  # use folium to add alternative tiles
folium.LayerControl().add_to(m_routes_intersection)  # use folium to add layer control

m_routes_intersection  # show map

In [ ]:
route_intersections

### Filtering data more to create one small instance

In [ ]:
filtered_int_geo_data = geo_data.cx[-9.43756:-9.413442,38.691779:38.705018]

In [ ]:
filtered_intersections = get_intersections(filtered_int_geo_data, 'Rua Id')

In [ ]:
bus_rou_gp

### Ploting only bus stops

In [ ]:
filtered_bus_stops = feed.stops

In [ ]:
filtered_bus_stops = filtered_bus_stops[(filtered_bus_stops['stop_lat'] >= 38.691779) & (filtered_bus_stops['stop_lat'] <= 38.705018)]

In [ ]:
filtered_bus_stops

In [ ]:
filtered_bus_stops = filtered_bus_stops[(filtered_bus_stops['stop_lon'] >= -9.43756) & (filtered_bus_stops['stop_lon'] <= -9.413442)]

In [ ]:
filtered_bus_stops

In [ ]:
geometry = [Point(xy) for xy in zip(filtered_bus_stops.stop_lon, filtered_bus_stops.stop_lat)]
df_bus_stops = filtered_bus_stops.drop(['stop_lon', 'stop_lat'], axis=1)
bus_stops_gj = gp.GeoDataFrame(df_bus_stops, crs="EPSG:4326", geometry=geometry)

In [ ]:
bus_stops_gj.explore()

### Checking the busstops with the street intersections

In [ ]:


fig, ax = plt.subplots(figsize = (15,15))
route_fig = filtered_int_geo_data.plot(ax = ax, color = 'darkgray', label = 'Routes')
intersect_points = filtered_intersections.plot(ax = ax, color = 'black', label = 'Route Intersections')
bus_stops = bus_stops_gj.plot(ax = ax, color = 'red', label = 'Bus Stops')
ax.legend(shadow=True, fancybox=True)
ax.set_title('Area of interest')
plt.show()

In [ ]:
m_routes_intersection = filtered_intersections.explore(color="black", legend = True)
filtered_int_geo_data.explore(m=m_routes_intersection, color="red")
folium.TileLayer('Stamen Toner', control=True).add_to(m_routes_intersection)  # use folium to add alternative tiles
folium.LayerControl().add_to(m_routes_intersection)  # use folium to add layer control

m_routes_intersection  # show map

### Making the distance matrix without Rewards

In [ ]:
filtered_int_geo_data.crs

In [ ]:
poly_line = ((-9.43529, 38.69845), (-9.43531, 38.69842), (-9.43534, 38.69840), (-9.43538, 38.69838), (-9.43542, 38.69835), (-9.43548, 38.69832), (-9.43603, 38.69803))
len_dist = 0
for i in range(6):
    len_dist += distance.distance((poly_line[i][0],poly_line[i][1]), (poly_line[i+1][0],poly_line[i+1][1])).km
print(len_dist)

In [ ]:
degrees2kilometers(0.0008558264062900338)

In [ ]:
distance_matrix = gp.GeoDataFrame()
distance_matrix['rua_id'] = None
distance_matrix['start_coord'] = None
distance_matrix['end_coord'] = None
distance_matrix['distance'] = None

In [ ]:
for index, row in filtered_int_geo_data.iterrows():
    
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    distance_matrix.at[index,'rua_id'] = row['Rua Id']
    distance_matrix.at[index,'start_coord'] = first_coord
    distance_matrix.at[index,'end_coord'] = last_coord
    distance_matrix.at[index,'distance'] = degrees2kilometers(row['geometry'].length)

In [ ]:
distance_matrix

### Comparing the linestring coordinates with street intersections

In [ ]:
coordinates = []
coordinates = distance_matrix.start_coord
geometry = [Point(point) for point in coordinates]
matrix = gp.GeoDataFrame(distance_matrix, crs="EPSG:4326", geometry=geometry)

In [ ]:
m_routes_intersection = route_intersections.explore(color="black", legend = True)
matrix.explore(m=m_routes_intersection, color="red")
folium.TileLayer('Stamen Toner', control=True).add_to(m_routes_intersection)  # use folium to add alternative tiles
folium.LayerControl().add_to(m_routes_intersection)  # use folium to add layer control

m_routes_intersection  # show map

In [ ]:
matrix.head()

### Finding out if bus stops are on edges



In [ ]:
filtered_int_geo_data.reset_index(drop=True, inplace=True)
bus_stops_gj.reset_index(drop=True, inplace=True)

In [ ]:
# functions to find out miinimum distance of bus stops from an edge

def closest_line(point):
    # get distances
    distance_list = [line.distance(point) for line in filtered_int_geo_data.geometry]
    shortest_distance = min(distance_list) # find the line closest to the point
    return(distance_list.index(shortest_distance), # return the closest line
           shortest_distance) # return the distance to that line

point = bus_stops_gj.geometry[0]
distance_list = [line.distance(point) for line in filtered_int_geo_data.geometry]

shortest_distance = min(distance_list)
distance_list.index(shortest_distance)

print(filtered_int_geo_data.geometry[60])

In [ ]:
# Update Matrix data frame with the availibility of bus stops on edges

matrix['bus_stop']=0
for bus_stop in bus_stops_gj.geometry:
    row_number, min_distance = closest_line(bus_stop)
    matrix.at[row_number,'bus_stop'] = 1
    
matrix['rewarded_distance']=matrix['distance']

for index, row in matrix.iterrows():
    if row['bus_stop']==1:
        matrix.at[index,'rewarded_distance']=row['distance']*0.8
        
matrix.head()

# Implementation of Dijkstra's Aglorithm for Shortest Path

In [ ]:
# converting GeodataFrame to pandas DataFrame

rtm = matrix.copy(deep=True)

rtm1 = pd.DataFrame(rtm[['start_coord','end_coord','distance']])
rtm1['distance'] = pd.to_numeric(rtm1['distance'],errors = 'coerce').round(5)

# converting to list for implementation of SPT Algorithm
edges = list(rtm1.itertuples(index=False, name=None))

In [ ]:
from collections import defaultdict

class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [ ]:
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
    
    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path

In [ ]:
# Adding edges to graph

graph = Graph()
for edge in edges:
    graph.add_edge(*edge)

In [ ]:
# (-9.4301681513, 38.6961235383)
# (-9.4296513887, 38.7040219405)


In [ ]:
# Finding the shortest path between start and end coordinates

shortest_route = dijsktra(graph,(-9.430754341, 38.694451877),(-9.4209469339, 38.7008490124))
shortest_route

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

polygon = LineString(shortest_route)

gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon])
gdf.explore()

In [ ]:
original_route = pd.DataFrame(rtm1['start_coord'])
original_route.head()
test = original_route['start_coord'].to_list()
test

In [ ]:
# polygon_original = LineString(test)

# gdf_original = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_original])
# #print(gdf_original)
# gdf_original.explore()

## Test by Removing a Segment

In [ ]:
# nodes to delete

# removed_section = [(-9.4293399588, 38.6975840454),(-9.427891414, 38.6987539024),(-9.4285920277, 38.6982331435)
#                       ,(-9.4268123772, 38.6995415008),(-9.4301681513, 38.6961235383)]


#removed_section = [(-9.4301681513, 38.6961235383)]

# removed_section = [(-9.4300212489, 38.7017111637),(-9.4302758283, 38.7021729478),(-9.4294039633, 38.702505643)]

removed_section = [(-9.4293399588, 38.6975840454),(-9.4301681513, 38.6961235383)]



In [ ]:
rtm2 = pd.DataFrame(rtm[['start_coord','end_coord','rewarded_distance']])
rtm2['rewarded_distance'] = pd.to_numeric(rtm2['rewarded_distance'],errors = 'coerce').round(5)

rtm_test = rtm2.copy(deep=True)
rtm_test.head()

In [ ]:
# removed the selected sections

for i in removed_section:
    rtm_test = rtm_test[rtm_test['end_coord']!=i]
    rtm_test = rtm_test[rtm_test['start_coord']!=i]

# convert the panda Dataframe to list
edges_test = list(rtm_test.itertuples(index=False, name=None))

In [ ]:
from collections import defaultdict


class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight
        
        
        
        
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
    
    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path

In [ ]:
graph_test = Graph()
for edge_test in edges_test:
    graph_test.add_edge(*edge_test)

In [ ]:
shortest_route_test = dijsktra(graph_test,(-9.430754341, 38.694451877),(-9.4209469339, 38.7008490124))
shortest_route_test

In [ ]:
polygon_test = LineString(shortest_route_test)

gdf_test = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_test])
m_test = gdf_test.explore(color="Blue")

gdf.explore(m=m_test, color="red")

In [ ]:
degrees2kilometers(polygon.length),degrees2kilometers(polygon_test.length)

In [ ]:
degrees2kilometers(polygon.length),degrees2kilometers(polygon_test.length)